In [ ]:
import pandas as pd
import os
import glob
import numpy as np

# Support Function

In [ ]:
def read_all_csv_in_directory(directory, recursive=False):
    """
    Reads all CSV files in a specified directory and combines them into a single DataFrame.

    Parameters:
    -----------
    directory : str
        Path to the directory containing the CSV files.
    recursive : bool, optional (default=False)
        If True, the function will search for CSV files recursively in subdirectories.

    Returns:
    --------
    pd.DataFrame
        A DataFrame containing the combined data from all CSV files.
    """
    all_files = []
    
    # Traverse the directory
    if recursive:
        # Use os.walk for recursive directory traversal
        for root, dirs, files in os.walk(directory):
            for file in files:
                if file.endswith('.csv'):
                    file_path = os.path.join(root, file)
                    print(f"Reading file: {file_path}")
                    df = pd.read_csv(file_path)
                    all_files.append(df)
    else:
        # List all files in the main directory
        for file in os.listdir(directory):
            if file.endswith('.csv'):
                file_path = os.path.join(directory, file)
                print(f"Reading file: {file_path}")
                df = pd.read_csv(file_path)
                all_files.append(df)
    
    # Concatenate all DataFrames
    if all_files:
        combined_df = pd.concat(all_files, ignore_index=True)
        return combined_df
    else:
        raise FileNotFoundError(f"No CSV files found in directory {directory}")
    

In [ ]:
def get_title(formatted_name, product_name):
    if ((formatted_name == 0) | (formatted_name == '') | (pd.isna(formatted_name))):
        result = product_name
    else:
        result = formatted_name
    return result

In [ ]:
def get_filter(title_x, title_y):
    if title_x == title_y:
        result = False
    else:
        result = True
    return result

# Read Morris data

In [ ]:
df_morris = pd.read_excel('data/All_Products_PWHSL.xlsx')

In [ ]:
df_morris.head()

In [ ]:
df_morris.shape

In [ ]:
df_morris['Flag'] = True

In [ ]:
df_morris['Title'] = df_morris.apply(lambda x: get_title(x['FormattedName'], x['ProductName']), axis=1)

In [ ]:
df_morris[df_morris.duplicated(subset='Title')][['Title', 'Flag']]

In [ ]:
df_morris.drop_duplicates(subset='Title', inplace=True)

In [ ]:
df_morris.shape

In [ ]:
df_morris[df_morris.duplicated('Sku')]

# Read Shopify data

In [ ]:
shopify_df = read_all_csv_in_directory('data/shopify')

In [ ]:
shopify_df.head()

In [ ]:
shopify_df.shape

In [ ]:
shopify_df['Variant Barcode'] = shopify_df['Variant Barcode'].str.replace("'", "")
shopify_df['Variant Barcode'] = shopify_df['Variant Barcode'].str.replace(' ', '')

In [ ]:
shopify_df['Variant Barcode'] = pd.to_numeric(shopify_df['Variant Barcode'], downcast='integer', errors='ignore')

In [ ]:
shopify_df['Variant Barcode'].fillna(0, inplace=True)

In [ ]:
shopify_df[shopify_df['Variant Barcode'] == 0].head()

In [ ]:
shopify_df[shopify_df['Variant Barcode'] == 0].shape

# Filter Morris product from shopify data

In [ ]:
shopify_morris_df = pd.merge(shopify_df, df_morris[['Title', 'Sku', 'Flag']], how='inner', left_on='Variant SKU', right_on='Sku')

In [ ]:
shopify_morris_df.head()

In [ ]:
shopify_morris_df.shape

# Duplicate data

In [ ]:
cleaned_df = shopify_morris_df[~pd.isna(shopify_morris_df['Variant SKU'])]

In [ ]:
cleaned_df.head()

In [ ]:
cleaned_df.shape

In [ ]:
cleaned_df[cleaned_df['Variant SKU'] == 'GC3582710']

In [ ]:
duplicated_df = cleaned_df[cleaned_df.duplicated(subset='Variant SKU', keep=False)].sort_values(by='Variant SKU')

In [ ]:
duplicated_df.shape

In [ ]:
duplicated_df.head()

In [ ]:
duplicated_df[duplicated_df['Variant SKU'] == 'GC3582710']

## Handle data with duplicated sku same product

In [ ]:
need_to_drop_df = duplicated_df[duplicated_df.duplicated(subset=['Variant SKU', 'Variant Barcode'], keep=False)].sort_values(by='Variant SKU')

In [ ]:
need_to_drop_df.head()

In [ ]:
need_to_drop_df.shape

In [ ]:
need_to_drop_df[need_to_drop_df['Variant SKU'] == 'GC3582710']

In [ ]:
need_to_drop_df['Filter'] = need_to_drop_df.apply(lambda x: get_filter(x['Title_x'], x['Title_y']), axis=1)

In [ ]:
filtered_need_to_drop = need_to_drop_df[need_to_drop_df['Filter']]

In [ ]:
filtered_need_to_drop[['Handle','Title_x', 'Variant SKU', 'Variant Barcode']]

In [ ]:
filtered_need_to_drop[need_to_drop_df['Variant SKU'] == 'GC3582710']

In [ ]:
filtered_need_to_drop.to_csv('data/need_to_drop.csv', index=False)

## Handle data with duplicated sku different product

In [ ]:
need_to_change_sku_df = duplicated_df.groupby('Variant SKU').filter(lambda x: x['Variant Barcode'].nunique() > 1)

In [ ]:
filtered_need_to_change_sku_df = need_to_change_sku_df.drop_duplicates(subset=['Variant SKU', 'Variant Barcode'])

In [ ]:
filtered_need_to_change_sku_df[['Handle', 'Title', 'Variant SKU', 'Variant Barcode']]

In [ ]:
merged_need_to_change_df = filtered_need_to_change_sku_df.merge(df_morris[['Title', 'Flag']], how='left', on='Title')

In [ ]:
merged_need_to_change_df

In [ ]:
need_to_change_sku_df.to_csv('data/need_to_change_sku.csv', index=False)